# Comparison between ClimateTRACE and C40 inventories

This analysis compares city aggregated estimates from [climateTRACE](https://climatetrace.org/) to estimates [GPC](https://ghgprotocol.org/ghg-protocol-cities) compliant C40 city inventories downloaded from [here](https://www.c40knowledgehub.org/s/article/C40-cities-greenhouse-gas-emissions-interactive-dashboard?language=en_US).

I am not sure if the C40 inventories are high quality. Comparing to downscaled observations would not be a fair comparison. 

In [33]:
import json
import os
import fnmatch
import pandas as pd
import tarfile
import os
import requests
from sqlalchemy import create_engine, MetaData, text
from sqlalchemy.orm import sessionmaker
from tqdm import tqdm

In [30]:
from utils import (
    get_c40_data, 
    filter_out_notation_keys,
    climatetrace_file_names,
    load_climatetrace_file,
    point_to_lat_lon,
    lat_lon_to_locode
)

## Read raw C40 data

**Units**: metric tonnes CO2-eq. (I am assuming these are units since they should be following the GPC)

In [47]:
df_c40_raw = get_c40_data()

### filter C40

In [64]:
refnos = ['I.4.1']
columns = ['city', 'locode', 'year'] + refnos

df_tmp = filter_out_notation_keys(df_c40_raw, refnos)
df_c40 = (
    df_tmp
    .loc[:, columns]
    .rename(columns = {'I.4.1': 'emissions_c40'})
)

### Read ClimateTRACE

**Units**: Units are tonnes 

In [25]:
asset_file = './fossil_fuel_operations/asset_oil-and-gas-refining_emissions.csv'
df_ct_raw = load_climatetrace_file(asset_file)
filt = (df_ct_raw['gas'] == 'co2e_100yr')
df_tmp = df_ct_raw.loc[filt]

In [34]:
points = set(df_tmp['st_astext'])
df_points = pd.DataFrame([point_to_lat_lon(point) for point in points])

df_merged = df_tmp.merge(df_points, on='st_astext')
df_unique = df_merged[['lon','lat']].drop_duplicates()
print(f"number unique assets: {len(df_unique)}")

db_uri = "postgresql://ccglobal:@localhost/ccglobal"
engine = create_engine(db_uri)
metadata_obj = MetaData()
Session = sessionmaker(bind=engine)
session = Session()

output_list = []
for _, row in tqdm(df_unique.iterrows()):
    lat = row['lat']
    lon = row['lon']
    locode = lat_lon_to_locode(session, lat, lon)
    output_list.append({'lon':lon, 'lat': lat, 'locode': locode})

session.close()

df_locodes = pd.DataFrame(output_list)
df_merged_locodes = df_merged.merge(df_locodes, on =['lat','lon'])
filt = df_merged_locodes['locode'].notnull()
df_data = df_merged_locodes[filt]

number unique assets: 592


592it [00:05, 110.57it/s]


In [70]:
df_ct = (
    df_data
    .assign(year = lambda row: pd.to_datetime(row['start_time']).dt.year)
    .loc[:, ['locode', 'year', 'emissions_quantity', 'emissions_factor_units']]
    .rename(columns = {'emissions_quantity': 'emissions_ct'})
)

In [74]:
# check the units
set(df_ct['emissions_factor_units'])

{'tonnes_gas_per_product_refined'}

### Comparison

In [66]:
df_int = pd.merge(df_ct, df_c40, on = ['year', 'locode'], how='inner')
df_int['diff'] = df_int['emissions_ct'] - df_int['emissions_c40']
df_int['percent_error'] = (df_int['diff'] / df_int['emissions_c40']) * 100

In [67]:
df_int

,locode,year,emissions_ct,city,emissions_c40,diff,percent_error
0,US LAX,2019,2.655844e+04,Los Angeles,4229906.827511,-4203348.386181,-99.372127
1,US LAX,2019,1.060106e+06,Los Angeles,4229906.827511,-3169800.348511,-74.937829
2,US LAX,2015,2.584065e+04,Los Angeles,897096.709239,-871256.063619,-97.119525
3,US LAX,2015,1.031455e+06,Los Angeles,897096.709239,134358.242761,14.977008
4,US LAX,2016,2.575092e+04,Los Angeles,900281.218994,-874530.297844,-97.13968
5,US LAX,2016,1.027874e+06,Los Angeles,900281.218994,127592.293006,14.172493
6,US LAX,2017,2.655844e+04,Los Angeles,968068.88132,-941510.43999,-97.256555
7,US LAX,2017,1.060106e+06,Los Angeles,968068.88132,92037.59768,9.50734
8,US LAX,2018,2.790431e+04,Los Angeles,4494093.539186,-4466189.230896,-99.379089
9,US LAX,2018,1.113828e+06,Los Angeles,4494093.539186,-3380265.448186,-75.215734
